In [ ]:
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
from textblob import TextBlob
from sklearn.preprocessing import LabelEncoder
import joblib as jb
import numpy as np

In [ ]:
df = pd.read_csv('../data/social_media.csv')

In [ ]:
df = df[df['platform'] == 'Instagram']
df.drop(columns=['platform'] , inplace=True)

In [ ]:
df['ER'] = ((df['comments_count'] + df['likes'] + df['shares']+ df['views'])/ df['follower_count']) * 100

In [ ]:
df.drop(df[df['ER'] > 100].index , inplace=True)


In [ ]:
mean = df['ER'].mean()
std = df['ER'].std()
high_limit = mean + 3 * std
low_limit = mean - 3 * std
high_limit , low_limit

df = df[df['ER'] <= high_limit]

In [ ]:
df['ER'].describe()
df['ER'].describe()
plt.figure(figsize=(6,4))
sns.boxplot(x=df['ER']/100)
plt.title('Boxplot du Taux d\'Engagement (ER)')
plt.savefig('boxplot_ER_reparition.png' ,  dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
df['day'] = pd.to_datetime(df['post_date']).dt.day_name()
df['hour'] = pd.to_datetime(df['post_date']).dt.hour

In [ ]:
def calculate_description_score(description):
    if pd.isna(description):
        return 0.0
    blob = TextBlob(str(description))
    return blob.sentiment.polarity

In [ ]:
def count_tags(h_str):
    if pd.isna(h_str) or h_str == "": return 0
    return len(str(h_str).split(','))

In [ ]:
def count_description_words(description):
    if pd.isna(description):
        return 0
    return len(str(description).split())

In [ ]:
df['description_score'] = df['content_description'].apply(calculate_description_score)
df['hashtag_count'] = df['hashtags'].apply(count_tags)
df['desciption_word_count'] = df['content_description'].apply(count_description_words)

In [ ]:
day_map = {
    'Monday': 0,
    'Tuesday': 1,
    'Wednesday': 2,
    'Thursday': 3,
    'Friday': 4,
    'Saturday': 5,
    'Sunday': 6
}

df['day_num'] = df['day'].map(day_map)
df['day_sin'] = np.sin(2 * np.pi * df['day_num'] / 7)
df['day_cos'] = np.cos(2 * np.pi * df['day_num'] / 7)


In [ ]:
df = pd.get_dummies(df, columns=['content_type'], drop_first=True)
bool_cols = df.select_dtypes(include='bool').columns
df[bool_cols] = df[bool_cols].astype(int)

In [ ]:
df.columns

In [ ]:
df.drop(columns=['post_date' , 'content_description' , 'likes' ,'views','shares' ,'hashtags','day_num' ,'day' , 'comments_count'] , inplace=True)

In [ ]:
df.to_csv('../data/final.csv' , index=False)